In [1]:
import pandas as pd
import numpy as np
import requests
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
df=pd.read_csv('../Input/netflix_titles.csv')

In [3]:
netflix=df[['type','title','country','release_year','duration','listed_in','description']]
netflix=netflix[netflix.type=='Movie']
netflix=netflix[netflix.country=='United States']
netflix=netflix[netflix.release_year>1995]
netflix=netflix.head(1000)

In [4]:
apiKey = os.getenv("API_IH")
rapidapiKey = os.getenv("RAPIDAPI")

In [5]:
def getID(titulo, key):
    try:    
        url = "https://movie-database-imdb-alternative.p.rapidapi.com/"
        querystring = {"page":"1","r":"json","s":titulo}
        headers = {'x-rapidapi-host': "movie-database-imdb-alternative.p.rapidapi.com",'x-rapidapi-key': key}
        response = requests.request("GET", url, headers=headers, params=querystring)
        response=(response.text)
        response=((response.replace(' ','').replace('"imdbID":'," ").replace(',"Type":',' ').split())[1]).replace('\"',"")
    except:pass
    return(response)

In [6]:
netflix['id']=[getID(x,rapidapiKey) for x in netflix['title']]

In [7]:
netflix = netflix[~(netflix.id == '{"Response":"False","Error":"Movie not found!"}')]
netflix = netflix[~(netflix.id == '{"Response":"False","Error":"Too many results."}')]

In [8]:
net=netflix

In [9]:
def getFromTMDB(id_tmdb,i,key):
    try:
        res = (requests.get(f"https://api.themoviedb.org/3/movie/{id_tmdb}?api_key={key}&language=en-US")).json()[f'{i}']
    except:
        res = 'NaN'
    return res

In [10]:
for i in ['popularity','vote_count','vote_average']:
    net[f'{i}'] = [getFromTMDB(x,i,apiKey) for x in net['id']]

In [11]:
net = net[~(netflix.listed_in == 'Nan')]

In [12]:
net.head()

,type,title,country,release_year,duration,listed_in,description,id,popularity,vote_count,vote_average
4,Movie,#realityhigh,United States,2017,99 min,Comedies,When nerdy high schooler Dani finally attracts...,tt6119504,20.758,870,6.4
20,Movie,Manhattan Romance,United States,2014,98 min,"Comedies, Independent Movies, Romantic Movies",A filmmaker working on a documentary about lov...,tt2608324,7.044,24,4.8
23,Movie,Stonehearst Asylum,United States,2014,113 min,"Horror Movies, Thrillers","In 1899, a young doctor arrives at an asylum f...",tt1772264,26.797,880,6.7
24,Movie,The Runner,United States,2015,90 min,"Dramas, Independent Movies",A New Orleans politician finds his idealistic ...,tt1790864,87.885,12504,7.1
25,Movie,6 Years,United States,2015,80 min,"Dramas, Independent Movies, Romantic Movies",As a volatile young couple who have been toget...,tt3799372,15.477,624,5.4


In [13]:
net.to_csv("../Output/NetflixFull.csv",index=False)